## Imports and setup

Change the contents in the next code cell if you're running locally or on the colab instance.

In [ ]:
import sys
ENVIRONMENT = None
GAMES_PATH = None
PURCHASES_PATH = None
PRICES_PATH = None
PLAYERS_PATH = None

# -------------
# SET THIS BLOCK 
# -------------

# detect environment
if 'google.colab' in sys.modules:
    ENVIRONMENT = 'colab' 
    print('Google Colab environment detected')
    from google.colab import drive
    drive.mount('/content/drive')

    # Set paths to data
    GAMES_PATH = "/content/drive/MyDrive/datasets/games.csv"
    PURCHASES_PATH = "/content/drive/MyDrive/datasets/purchased_games.csv"
    PRICES_PATH = "/content/drive/MyDrive/datasets/prices.csv"
    PLAYERS_PATH = "/content/drive/MyDrive/datasets/players.csv"

    GAMES_ENCODED_PATH = "/content/drive/MyDrive/datasets/games_encoded.csv"
    GAMES_PRICES_MERGED_PATH = "/content/drive/MyDrive/datasets/2_games_prices_merged.csv"
    PRICE_FEATURES_PATH = "/content/drive/MyDrive/datasets/2_price_features.csv"
    PURCHASE_FEATURES_PATH = "/content/drive/MyDrive/datasets/3_purchase_features.csv"

else:
    ENVIRONMENT = 'local'
    print('Local environment detected')

    ###################################
    # CHANGE THIS PART
    ###################################

    # raw datasets
    GAMES_PATH = ''
    PURCHASES_PATH = ''
    PRICES_PATH = ''
    PLAYERS_PATH = ''
    
    # outputs
    GAMES_ENCODED_PATH = '/datasets/games_encoded.csv'
    GAMES_PRICES_MERGED_PATH = '/datasets/2_games_prices_merged.csv'
    PRICE_FEATURES_PATH = '/datasets/2_price_features.csv'
    PURCHASE_FEATURES_PATH = '/datasets/3_purchase_features.csv'
    
    ##################################


assert(ENVIRONMENT in ['local', 'colab']), "ENVIRONMENT variable must be either 'local' or 'colab'"
assert(GAMES_PATH is not None and GAMES_PATH != ''), "GAMES_PATH variable must be set"
assert(PURCHASES_PATH is not None and PURCHASES_PATH != ''), "PURCHASES_PATH variable must be set"
assert(PRICES_PATH is not None and PRICES_PATH != ''), "PRICES_PATH variable must be set"
assert(PLAYERS_PATH is not None and PLAYERS_PATH != ''), "PLAYERS_PATH variable must be set"
print(f"Using data paths:\nGAMES_PATH: {GAMES_PATH}\nPURCHASES_PATH: {PURCHASES_PATH}\nPRICES_PATH: {PRICES_PATH}\nPLAYERS_PATH: {PLAYERS_PATH}")

Local environment detected
Using data paths:
GAMES_PATH: /datasets/games.csv
PURCHASES_PATH: /datasets/purchases.csv
PRICES_PATH: /datasets/prices.csv
PLAYERS_PATH: /datasets/players.csv


In [ ]:
%pip install tqdm
%pip install yfinance
%pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
from google.colab import drive

In [ ]:
df_games = pd.read_csv(GAMES_PATH)
df_purchases = pd.read_csv(PURCHASES_PATH)
df_prices = pd.read_csv(PRICES_PATH)
df_players = pd.read_csv(PLAYERS_PATH)
# df_reviews = pd.read_csv("/content/drive/MyDrive/datasets/reviews.csv")


In [ ]:
dfs = [df_games, df_purchases, df_prices, df_players]
for df in dfs:
  print(df.columns)

Index(['gameid', 'title', 'developers', 'publishers', 'genres',
       'supported_languages', 'release_date'],
      dtype='object')
Index(['playerid', 'library'], dtype='object')
Index(['gameid', 'usd', 'eur', 'gbp', 'jpy', 'rub', 'date_acquired'], dtype='object')
Index(['playerid', 'country', 'created'], dtype='object')


## 1.0 Games
This segment handles 2 features related to the `games` dataset.

Feature Name|Description
---|---
genres (one-hot encoded)|Vectorised representation of the genres associated with a game.<br>Each unique genre will be added as a new column
release_age_days|Captures the age of a game since its release date (int)

### 1.1 Data validation
Before creating the columns representing the genres, we need to first check what the format of the `genres` column is.

A quick look at the data suggests that the `genres` column is represented by a string-ified version of a comma-delimited array using single quotes for each element (i.e. `['genreA', 'genreB']`).

Apart from `NaN`s, we check to ensure that there are no other formats or incorrectly formatted rows.

In [ ]:
df_games[df_games['genres'].notnull()].head()

,gameid,title,developers,publishers,genres,supported_languages,release_date
5,3278740,NEURO,['Revolt Games'],['Strategy First'],['Action'],"['English', 'Russian']",2024-10-11
10,3270850,Keep Your Eyes Open,['Texerikus'],['Texerikus'],['Indie'],['English'],2024-10-21
15,3267350,Tiny Shooters,['madilumar'],['Thetinyverse'],"['Action', 'Free To Play']",['English'],2024-10-19
17,3266470,Futanari Sex Adventures - Episode 5,['EroticGamesClub'],['EroticGamesClub'],"['Adventure', 'Casual', 'Indie']",['English'],2024-10-17
20,3264110,AUTO_BATTLER_RPG,['Kadragon Games'],['Kadragon Games'],"['Casual', 'RPG']",['English'],2024-10-22


In [ ]:
# Checking for presence of edge cases
# Add helper col to show whether it matches expected format or not

non_null_genres = df_games['genres'].dropna()
print(f'Number of non-nulls: {len(non_null_genres)}')

# Find unusual representations
def detect_genre_format(g):
  '''
  Detects the format of the genres column.

  Assumption: Each value in the genres column is either NaN or wrapped in an array.
  e.g. ['genreA', 'genreB']
  This means they are
  - list-like
  - wrapped in single qoutes
  - comma-delimited
  '''
  if pd.isna(g):
    return 'NaN'
  # drop whitespace
  g = str(g).strip()

  # edge case if dropna() fails
  if g == '' or g.lower() == 'nan':
    return 'EmptyString'

  # assumption: ['genreA','genreB', ...]
  # list-like, each genre wrapped in single quote, comma-delimited
  elif g.startswith('[') and g.endswith(']'):
    return 'ListLike'
  elif ',' in g:
    return 'CommaSeparated'
  else:
    return 'Other'

df_games['genre_format'] = df_games['genres'].apply(detect_genre_format)

# Summarise types of representations found
print("Genre summary:")
print(df_games['genre_format'].value_counts())

# print examples of each non-standard format (if exists)
non_standard = df_games[df_games['genre_format'].isin(['CommaSeparated', 'Other', 'EmptyString'])]
print("\nExamples of non-standard genre representations:")
display(non_standard[['title', 'genres']].head(10))

# ensure that genres can be safely parsed
def safe_parse_genres(g):
  """
  Tries to parse values in the genre columns as pythonic lists
  Returns an empty list for invalid or NaN entries.
  """
  if pd.isna(g):
    return []
  g = str(g).strip()
  if not g:
    return []
  try:
    parsed = ast.literal_eval(g)
    if isinstance(parsed, list):
      return [s.strip() for s in parsed]
    elif isinstance(parsed, str):
      print(f'String representation - {g}')
      return [parsed.strip()]
    else:
      return []
  except (ValueError, SyntaxError) as e:
    print(f"Error parsing genres: {e}")
    return []

df_games['genres_cleaned'] = df_games['genres'].apply(safe_parse_genres)

# Check that all cleaned values are lists
assert df_games['genres_cleaned'].apply(
    lambda x: isinstance(x, list)).all(), "Some rows were not parsed into lists."
print("\nAll elems parsed as lists.")

# Remove helper col
df_games.drop(columns=['genre_format'], inplace=True)


Number of non-nulls: 92699
Genre summary:
genre_format
ListLike    92699
NaN          5549
Name: count, dtype: int64

Examples of non-standard genre representations:


,title,genres



All elems parsed as lists.


### 1.2 Features

Since the previous step showed that the dataset is as expected, we can now parse the dataset and begin adding features.



#### 1.2.1 Genre encoding


To add new columns for each genre, we first create a set representing unique genres, then create a binary column for each genre.

Since the `genres` column is multi-label, we use sklearn's [MultiLabelBinarizer (MLB)](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html) method for encoding.



In [ ]:

mlb_genres = MultiLabelBinarizer()

genre_matrix = mlb_genres.fit_transform(df_games['genres_cleaned'])

print(f'Genre set with {len(mlb_genres.classes_)} values: ')
print(mlb_genres.classes_)
print()

# Store matrix as dataframe and combine it with the games dataframe
# We then store it as a new variable so that the original df is untouched
genre_df = pd.DataFrame(genre_matrix, columns=mlb_genres.classes_, index=df_games.index)

df_games_encoded = pd.concat([df_games, genre_df], axis=1)

df_games_encoded.head()


Genre set with 33 values: 
['360 Video' 'Accounting' 'Action' 'Adventure' 'Animation & Modeling'
 'Audio Production' 'Casual' 'Design & Illustration' 'Documentary'
 'Early Access' 'Education' 'Episodic' 'Free To Play' 'Game Development'
 'Gore' 'Indie' 'Massively Multiplayer' 'Movie' 'Nudity' 'Photo Editing'
 'RPG' 'Racing' 'Sexual Content' 'Short' 'Simulation' 'Software Training'
 'Sports' 'Strategy' 'Tutorial' 'Utilities' 'Video Production' 'Violent'
 'Web Publishing']



,gameid,title,developers,publishers,genres,supported_languages,release_date,genres_cleaned,360 Video,Accounting,...,Short,Simulation,Software Training,Sports,Strategy,Tutorial,Utilities,Video Production,Violent,Web Publishing
0,3281560,Horror Game To Play With Friends! Playtest,NaN,NaN,NaN,NaN,2024-10-21,[],0,0,...,0,0,0,0,0,0,0,0,0,0
1,3280930,Eternals' Path Playtest,NaN,NaN,NaN,NaN,2024-10-17,[],0,0,...,0,0,0,0,0,0,0,0,0,0
2,3280770,ANGST: A TALE OF SURVIVAL - Singleplayer Playtest,NaN,NaN,NaN,NaN,2024-10-13,[],0,0,...,0,0,0,0,0,0,0,0,0,0
3,3279790,Montabi Playtest,NaN,NaN,NaN,NaN,2024-10-13,[],0,0,...,0,0,0,0,0,0,0,0,0,0
4,3278320,파이팅걸 유리 Playtest,NaN,NaN,NaN,NaN,2024-10-12,[],0,0,...,0,0,0,0,0,0,0,0,0,0


#### 1.2.2 Release age

We now estimate the age of a game since its release date, using a benchmark day of 1 October 2025.

In [ ]:
REFERENCE_DATE = pd.to_datetime('2025-10-01')
df_games_encoded['release_date'] = pd.to_datetime(df_games_encoded['release_date'])

# caclulate difference in days
df_games_encoded['release_age_days'] = (REFERENCE_DATE - df_games_encoded['release_date']).dt.days
df_games_encoded[['title', 'release_date', 'release_age_days']].head()

,title,release_date,release_age_days
0,Horror Game To Play With Friends! Playtest,2024-10-21,345
1,Eternals' Path Playtest,2024-10-17,349
2,ANGST: A TALE OF SURVIVAL - Singleplayer Playtest,2024-10-13,353
3,Montabi Playtest,2024-10-13,353
4,파이팅걸 유리 Playtest,2024-10-12,354


In [ ]:
# check for any possible issues
# - nulls
# - any future release dates - should not occur since this dataset should be historical

invalid_dates = df_games_encoded[df_games_encoded['release_date'].isna()]
future_dates = df_games_encoded[df_games_encoded['release_age_days'] < 0]

if not invalid_dates.empty:
  print(f'Examples of invalid date entries')
  print(invalid_dates[['title', 'release_date']].head())
if not future_dates.empty:
  print(f'Examples of future date entries')
  print(future_dates[['title', 'release_date']].head())

In [ ]:
# save dataset as csv
if ENVIRONMENT == 'colab':
    drive.mount('/content/drive')

df_games_encoded.to_csv(GAMES_ENCODED_PATH, index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2.0 Prices

2 more features are created using only the `prices` dataset. The dataset contains historical data of a game's prices.

Since the dataset captures a set date range of a game's price history, we shall assume that the highest price a game was ever at is its normal, full price.

We shall use this to determine the following features:

Feature Name|Description
---|---
price_volatility|The SD of the game's price, using its price as the base. This helps to later estimate how much a price can deviate over time
avg_discount|Captures the mean of all discounts that the game has had.<br><br>This assumes that the highest price the game was ever at is the base price,<br>due to dataset limitations that don't track whether a game increases its base price.

___

Before creating the features, we first handle prices and currency conversion.

The `prices` dataset contains columns for the games' prices for different currencies. We treat these prices as follows:
- If all prices are null, the game is assumed to be free
- Otherwise, we use the USD price of the game to create the above features. If the USD price is unavailable, we convert it to USD from the available currency(ies).

The converted currencies will be stored in the `price_usd` column.

### 2.1 Currency Conversion

To get historical fx rates, we use the ~[`currencyconverter`](https://pypi.org/project/CurrencyConverter/)~ [`Yahoo Finance`](https://pypi.org/project/yfinance/) library.

Steam allows publishers to assign different prices for different regions. This means that for a game with multiple non-USD prices available, the prices may all diverge after converting to USD.

If multiple converted prices exist, we will use the median of the converted values.

<u>Rules</u>

Scenario|Decision
---|---
All prices null|Game is free
USD price available|Take only the USD price
USD price unavailable &<br>**One** other currency available|Convert to USD
USD price unavailable & <br>**multiple** currencies available|Convert **all** to USD and take the median


**Note: `currencyconverter` is not used as the European Central Bank stopped publishing rates in 2022 for RUB due to the Russia-Ukraine conflict. For consistency, we will take all rates from yfinance.**

Fetching data from Yahoo Finance to create a lookup dictionary

In [ ]:
import yfinance as yf
# get the oldest date_acqured
df_prices['date_acquired'] = pd.to_datetime(df_prices['date_acquired'])
earliest_date = df_prices['date_acquired'].min()
# retrieve 1 week of earlier data to handle potential missing rows from yfinance
earliest_date -= pd.Timedelta(days=7)
earliest_date = earliest_date.date()

# get newest date_acquired
newest_date = df_prices['date_acquired'].max()
# we add 1 day here since yfinance doesn't include the end date
newest_date = newest_date.date() + pd.Timedelta(days=1)

eur_usd = yf.download('EURUSD=X', start=earliest_date, end=newest_date)['Close']
gbp_usd = yf.download('GBPUSD=X', start=earliest_date, end=newest_date)['Close']
jpy_usd = yf.download('JPYUSD=X', start=earliest_date, end=newest_date)['Close']
rub_usd = yf.download('RUBUSD=X', start=earliest_date, end=newest_date)['Close']

# print head
print(eur_usd.head())
print(gbp_usd.head())
print(jpy_usd.head())
print(rub_usd.head())

/tmp/ipython-input-4154375860.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  eur_usd = yf.download('EURUSD=X', start=earliest_date, end=newest_date)['Close']
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-4154375860.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  gbp_usd = yf.download('GBPUSD=X', start=earliest_date, end=newest_date)['Close']
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-4154375860.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  jpy_usd = yf.download('JPYUSD=X', start=earliest_date, end=newest_date)['Close']
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-4154375860.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  rub_usd = yf.download('RUBUSD=X', start=earliest_date, end=newest_date)['Close']
[

Ticker      EURUSD=X
Date                
2024-11-21  1.054619
2024-11-22  1.046934
2024-11-25  1.047987
2024-11-26  1.044430
2024-11-27  1.049120
Ticker      GBPUSD=X
Date                
2024-11-21  1.265534
2024-11-22  1.258479
2024-11-25  1.259382
2024-11-26  1.253306
2024-11-27  1.257387
Ticker      JPYUSD=X
Date                
2024-11-21  0.006439
2024-11-22  0.006484
2024-11-25  0.006487
2024-11-26  0.006478
2024-11-27  0.006536
Ticker      RUBUSD=X
Date                
2024-11-21  0.009941
2024-11-22  0.009872
2024-11-25  0.009643
2024-11-26  0.009614
2024-11-27  0.009479


#### 2.1.1 (outdated) Function based approach
Originally, we created a function to apply to each row of the dataframe to retrieve FX rates and conduct currency conversion. However, due to performance issues (4m operations for **approx 44h runtime**), we modifed it for a vectorised approach. Original codeblock left here for reference.

**Old code**


```python
# Store fx data in a dictionary for lookup
fx_data = {
    'eur': eur_usd['Close'],
    'gbp': gbp_usd['Close'],
    'jpy': jpy_usd['Close'],
    'rub': rub_usd['Close']
}

def get_fx_rate(currency, date):
  '''
  Fetches currency => USD exchange rate for a given date.
  '''
  series = fx_data[currency.lower()]
  # find the idx of the correct row from the fx series
  loc = series.index.searchsorted(date, side='right') - 1 # -1 for indexing
  if loc < 0:
    return np.nan
  
  rate = series.iloc[loc]
  # Error handling
  if isinstance(rate, (pd.Series, np.ndarray)):
      rate = rate.iloc[0] if hasattr(rate, 'iloc') else rate[0]
  return float(rate)

####### Implementation

currencies = ['usd', 'eur', 'gbp', 'jpy', 'rub']

# Case 1: Free games
nans_mask = df_prices[currencies].isna().all(axis=1)
df_prices.loc[nans_mask, 'price_usd'] = 0.0

# Case 2: default to USD if available
usd_mask = df_prices['usd'].notna()
df_prices.loc[usd_mask, 'price_usd'] = df_prices.loc[usd_mask, 'usd']

# case 3: non-free, no USD available

def convert_to_usd(row: pd.Series) -> float :
  '''
  Converts non-USD currecnies to USD using prevailing fx rates as at date_acquired.

  If > 1 non-USD currency available, returns the median of the converted values.
  '''
  # skip rows matching cases 1 / 2
  # compare the row's `name` with the index of the masked df
  if row.name in df_prices.index[nans_mask | usd_mask]:
    return row['price_usd']

  row_date = row['date_acquired']
  converted_values = [] # store all converted values in the array to get median later
  currencies = ['eur', 'gbp', 'jpy', 'rub']

  for currency in currencies:
    val = row[currency]
    if pd.notna(val):
      fx_rate = get_fx_rate(currency, row_date)
      if fx_rate is not None and not pd.isna(fx_rate):
        # USD: fx * price
        converted_values.append(fx_rate * val)
  
  if len(converted_values) == 0:
    return np.nan
  else:
    return np.median(converted_values)

# apply conversion
tqdm.pandas(desc='Converting currencies')
df_prices['price_usd'] = df_prices.progress_apply(convert_to_usd, axis=1)

```

#### 2.1.2 Vectorised approach

Rather than applying a function to all 4m rows, we use vectorised operations to benefit from numpy's optimisations.

This approach creates multiple helper columns in the `df_prices` dataset to store the FX rates and conduct row-wise calculations to convert the currencies.

In [ ]:
#####################################
# Prep prices dataframe with fx data#
#####################################
eur_usd = eur_usd.squeeze()
gbp_usd = gbp_usd.squeeze()
jpy_usd = jpy_usd.squeeze()
rub_usd = rub_usd.squeeze()
fx_data = pd.DataFrame({
    'date': eur_usd.index.normalize(), # handle potential time mismatche
    'eur_usd': eur_usd.values,
    'gbp_usd': gbp_usd.values,
    'jpy_usd': jpy_usd.values,
    'rub_usd': rub_usd.values
})

# drop nan days (if any)
fx_data.dropna(subset=['eur_usd', 'gbp_usd', 'jpy_usd', 'rub_usd'], how='all', inplace=True)
fx_data.sort_values('date', inplace=True)
fx_data.reset_index(drop=True, inplace=True)


# align dates with FX data and normalize datetimes to prevent time mismatch
df_prices['date_acquired'] = pd.to_datetime(df_prices['date_acquired']).dt.normalize()
df_prices.sort_values('date_acquired', inplace=True)

# merge price with fx
df_prices = pd.merge_asof(
    df_prices,
    fx_data,
    left_on='date_acquired',
    right_on='date',
    direction='backward'
)
# drop duped column
df_prices.drop(columns=['date'], inplace=True)
df_prices.head()

,gameid,usd,eur,gbp,jpy,rub,date_acquired,eur_usd,gbp_usd,jpy_usd,rub_usd
0,3281560,NaN,NaN,NaN,NaN,NaN,2024-11-28,1.056245,1.267765,0.006608,0.008839
1,1648550,NaN,NaN,NaN,NaN,NaN,2024-11-28,1.056245,1.267765,0.006608,0.008839
2,1648460,0.99,0.79,0.79,100.0,30.0,2024-11-28,1.056245,1.267765,0.006608,0.008839
3,1648450,2.99,NaN,2.09,310.0,82.0,2024-11-28,1.056245,1.267765,0.006608,0.008839
4,1648440,NaN,NaN,NaN,NaN,NaN,2024-11-28,1.056245,1.267765,0.006608,0.008839


In [ ]:
################################
# Implementation of conversion #
################################


currencies = ['usd', 'eur', 'gbp', 'jpy', 'rub']

# Case 1: Free games
nans_mask = df_prices[currencies].isna().all(axis=1)
df_prices.loc[nans_mask, 'price_usd'] = 0.0

# Case 2: default to USD if available
usd_mask = df_prices['usd'].notna()
df_prices.loc[usd_mask, 'price_usd'] = df_prices.loc[usd_mask, 'usd']

# case 3: non-free, no USD available
tqdm.pandas()
t_pbar = tqdm(total=1)
t_pbar.set_description(desc="Converting currencies")
converted= pd.concat([
    df_prices['eur'] * df_prices['eur_usd'],
    df_prices['gbp'] * df_prices['gbp_usd'],
    df_prices['jpy'] * df_prices['jpy_usd'],
    df_prices['rub'] * df_prices['rub_usd']
], axis=1)

# handle median
case3_mask = df_prices['price_usd'].isna() # mask on price_usd column for all remaining unhandled rows
df_prices.loc[case3_mask, 'price_usd'] = converted[case3_mask].median(axis=1, skipna=True)
t_pbar.update(1)
t_pbar.close()

# print
print()
print(f'Rows missing price_usd after conversion: {df_prices["price_usd"].isna().sum()}')

display(df_prices.head())

Converting currencies: 100%|██████████| 1/1 [00:00<00:00,  2.43it/s]


Rows missing price_usd after conversion: 0


,gameid,usd,eur,gbp,jpy,rub,date_acquired,eur_usd,gbp_usd,jpy_usd,rub_usd,price_usd
0,3281560,NaN,NaN,NaN,NaN,NaN,2024-11-28,1.056245,1.267765,0.006608,0.008839,0.00
1,1648550,NaN,NaN,NaN,NaN,NaN,2024-11-28,1.056245,1.267765,0.006608,0.008839,0.00
2,1648460,0.99,0.79,0.79,100.0,30.0,2024-11-28,1.056245,1.267765,0.006608,0.008839,0.99
3,1648450,2.99,NaN,2.09,310.0,82.0,2024-11-28,1.056245,1.267765,0.006608,0.008839,2.99
4,1648440,NaN,NaN,NaN,NaN,NaN,2024-11-28,1.056245,1.267765,0.006608,0.008839,0.00


In [ ]:
# cleanup: remove helper fx columns

df_prices = df_prices.drop(columns=['eur_usd', 'gbp_usd', 'jpy_usd', 'rub_usd'])


### 2.2 Features - Price Volatility & Avg discounts

This is a measure of the standard deviation of a game's price over time, given by $std(\frac{price_{i}}{base_{price}})$

We group the dataset by `gameid`. For a given game, we first find the normal full price (highest recorded price), before calculating the discount factor for all other prices recorded.

Using the discount factor allows us to calculate average discount for a game and its standard deviation.

In [ ]:
df_prices.head()

,gameid,usd,eur,gbp,jpy,rub,date_acquired,price_usd
0,3281560,NaN,NaN,NaN,NaN,NaN,2024-11-28,0.00
1,1648550,NaN,NaN,NaN,NaN,NaN,2024-11-28,0.00
2,1648460,0.99,0.79,0.79,100.0,30.0,2024-11-28,0.99
3,1648450,2.99,NaN,2.09,310.0,82.0,2024-11-28,2.99
4,1648440,NaN,NaN,NaN,NaN,NaN,2024-11-28,0.00


In [ ]:
# Populate df with the full price of each game
base_price_df = (
    df_prices
    .groupby('gameid', as_index=False)['price_usd']
    .max()
    .rename(columns={'price_usd': 'base_price'})
)
df_prices = pd.merge(df_prices, base_price_df, on='gameid', how='left')

# Calculate discount factors; handle free games
df_prices['discount_factor'] = df_prices['price_usd'] / df_prices['base_price']
df_prices.loc[df_prices['base_price'] == 0.0, 'discount_factor'] = 1

tqdm.pandas(desc="Computing price-based features")
price_features = (
    df_prices
    .groupby('gameid', as_index=False)
    .progress_apply(lambda x: pd.Series({
        'base_price': x['base_price'].iloc[0],
        'price_volatility': x['price_usd'].std(skipna=True),
        'avg_discount': 1-x['discount_factor'].mean(skipna=True)
    }))
    .reset_index()
)

print()
print(f'Number of games with single price point: {len(price_features[price_features["price_volatility"].isna()])}')
print(f'Missing avg discount: {(price_features["avg_discount"].isna()).sum():,}')

display(price_features.head())

Computing price-based features: 100%|██████████| 98465/98465 [01:11<00:00, 1377.73it/s]


Number of games with single price point: 0
Missing avg discount: 0


,index,gameid,base_price,price_volatility,avg_discount
0,0,10,9.99,3.476908,0.195751
1,1,20,4.99,1.629801,0.183701
2,2,30,4.99,1.629801,0.183701
3,3,40,4.99,1.629801,0.183701
4,4,50,4.99,1.629801,0.183701


### 2.3 Merging with games dataset

Now that the prices, SD and average discounts have been calculated - we merge them back into the `games` dataset created in 1.0

In [ ]:
# load variable in case the runtime was restarted
df_games_encoded = pd.read_csv(GAMES_ENCODED_PATH)
print(price_features.columns)
print(df_games_encoded.columns)

Index(['index', 'gameid', 'base_price', 'price_volatility', 'avg_discount'], dtype='object')
Index(['gameid', 'title', 'developers', 'publishers', 'genres',
       'supported_languages', 'release_date', 'genres_cleaned', '360 Video',
       'Accounting', 'Action', 'Adventure', 'Animation & Modeling',
       'Audio Production', 'Casual', 'Design & Illustration', 'Documentary',
       'Early Access', 'Education', 'Episodic', 'Free To Play',
       'Game Development', 'Gore', 'Indie', 'Massively Multiplayer', 'Movie',
       'Nudity', 'Photo Editing', 'RPG', 'Racing', 'Sexual Content', 'Short',
       'Simulation', 'Software Training', 'Sports', 'Strategy', 'Tutorial',
       'Utilities', 'Video Production', 'Violent', 'Web Publishing',
       'release_age_days'],
      dtype='object')


In [ ]:
df_games_merged = df_games_encoded.merge(
  price_features[[ 'gameid', 'base_price', 'price_volatility', 'avg_discount']]
)

display(df_games_merged.head())

,gameid,title,developers,publishers,genres,supported_languages,release_date,genres_cleaned,360 Video,Accounting,...,Strategy,Tutorial,Utilities,Video Production,Violent,Web Publishing,release_age_days,base_price,price_volatility,avg_discount
0,3281560,Horror Game To Play With Friends! Playtest,NaN,NaN,NaN,NaN,2024-10-21,[],0,0,...,0,0,0,0,0,0,345,0.0,0.0,0.0
1,3280930,Eternals' Path Playtest,NaN,NaN,NaN,NaN,2024-10-17,[],0,0,...,0,0,0,0,0,0,349,0.0,0.0,0.0
2,3280770,ANGST: A TALE OF SURVIVAL - Singleplayer Playtest,NaN,NaN,NaN,NaN,2024-10-13,[],0,0,...,0,0,0,0,0,0,353,0.0,0.0,0.0
3,3279790,Montabi Playtest,NaN,NaN,NaN,NaN,2024-10-13,[],0,0,...,0,0,0,0,0,0,353,0.0,0.0,0.0
4,3278320,파이팅걸 유리 Playtest,NaN,NaN,NaN,NaN,2024-10-12,[],0,0,...,0,0,0,0,0,0,354,0.0,0.0,0.0


In [ ]:
print(f'Missing base price: {df_games_merged["base_price"].isna().sum():,}')
print(f'Missing price volatility: {df_games_merged["price_volatility"].isna().sum():,}')
print(f'Missing avg discount: {df_games_merged["avg_discount"].isna().sum():,}')

Missing base price: 0
Missing price volatility: 0
Missing avg discount: 0


In [ ]:
# save as csv
if ENVIRONMENT == 'colab': 
    drive.mount('/content/drive')
    
df_games_merged.to_csv(GAMES_PRICES_MERGED_PATH, index=False)
price_features.to_csv(PRICE_FEATURES_PATH, index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 3.0 Purchases

We create two more features using the `purchases` dataset.


Feature Name|Description
---|---
library_size|A count of all unique games in a player's library
avg_purchase_price|Looks up the prices calculated in step 2.0, then<br>aggregates and gets the mean across all owned games


In [ ]:
# load previously created datasets (if runtime is restarted)

if ENVIRONMENT == 'colab':
    drive.mount('/content/drive')

df_games_merged = pd.read_csv(GAMES_PRICES_MERGED_PATH)
price_features = pd.read_csv(PRICE_FEATURES_PATH)

### 3.1 Data validation

Before creating features, we check the dataset to validate that the format for the `library` column is consistent throughout the dataset, similar to section 1.1.

In [ ]:
print(f'Total number of elements: {df_purchases.shape[0]}')
# nulls
print(f'Nulls in library column: {df_purchases["library"].isna().sum()}')

# ensure that library's non-null contents all resemble an array of ints
def detect_library_format(x):
  '''
  Detects the format of the library column. Tests out parsing the
  contents of the library column into a list of ints.

  Checks whether the contents are
  - NaN
  - blank or stringified NaN (i.e. x = 'NaN' instead of x=np.nan)
  - valid - string representing an array of ints
  Throws if value is malformed
  '''

  if pd.isna(x):
    return 'NaN'

  # drop whitespace
  x = str(x).strip()

  # if dropna didn't pick up
  if x == '' or x.lower() == 'nan':
    return "EmptyString"
  elif x.startswith('[') and x.endswith(']'):
    try:
      x = ast.literal_eval(x)
      if isinstance(x, list):
        # check what format the elements are
        if all(isinstance(i, int) for i in x):
          return 'Valid'
        else:
          return 'InvalidElements'
      else:
        return 'InvalidFormat'
    except ValueError:
      return 'InvalidSyntax'
  else:
    return 'Other'

# temproary new df with a copy of the library column
df_temp = df_purchases[['library']].copy()
df_temp['format'] = df_temp['library'].apply(detect_library_format)

# Summarise types of representation
print(f'`library` format summary:')
print(df_temp['format'].value_counts())

# show if there are any non standard cases (not Valid and not NaN)
print(f'\nExamples of non-standard entries:')
non_standard = df_temp[df_temp['format'].isin(['EmptyString', 'InvalidElements', 'InvalidSyntax', 'Other'])]
display(non_standard.head())


Total number of elements: 102548
Nulls in library column: 55607
`library` format summary:
format
NaN      55607
Valid    46941
Name: count, dtype: int64

Examples of non-standard entries:


,library,format


### 3.2 Features

Based on the previous step, all rows contain either NaNs or string representations of arrays.

Since all rows are valid, we then parse the contents of the `library` column.

In [ ]:
tqdm.pandas(desc='Parsing library column')
def parse_library(x):
  if pd.isna(x):
    return []
  return ast.literal_eval(x)

df_purchases['library_list'] = df_purchases['library'].progress_apply(parse_library)

df_purchases.head()

Parsing library column: 100%|██████████| 102548/102548 [00:33<00:00, 3053.50it/s]


,playerid,library,library_list
0,76561198060698936,"[60, 1670, 3830, 1600, 2900, 2910, 2920, 4800,...","[60, 1670, 3830, 1600, 2900, 2910, 2920, 4800,..."
1,76561198287452552,"[10, 80, 100, 240, 2990, 6880, 6910, 6920, 698...","[10, 80, 100, 240, 2990, 6880, 6910, 6920, 698..."
2,76561198040436563,"[10, 80, 100, 300, 20, 30, 40, 50, 60, 70, 130...","[10, 80, 100, 300, 20, 30, 40, 50, 60, 70, 130..."
3,76561198042412488,"[300, 240, 220, 320, 360, 4300, 4800, 4000, 61...","[300, 240, 220, 320, 360, 4300, 4800, 4000, 61..."
4,76561198119605821,"[47870, 108600, 550, 271590, 331470, 381210, 2...","[47870, 108600, 550, 271590, 331470, 381210, 2..."


#### 3.2.1 Library Size
Using the lists in `library_list`, we compute the number of games that each user has.

In [ ]:
df_purchases['library_size'] = df_purchases['library_list'].apply(len)

df_purchases.head()

,playerid,library,library_list,library_size
0,76561198060698936,"[60, 1670, 3830, 1600, 2900, 2910, 2920, 4800,...","[60, 1670, 3830, 1600, 2900, 2910, 2920, 4800,...",3340
1,76561198287452552,"[10, 80, 100, 240, 2990, 6880, 6910, 6920, 698...","[10, 80, 100, 240, 2990, 6880, 6910, 6920, 698...",476
2,76561198040436563,"[10, 80, 100, 300, 20, 30, 40, 50, 60, 70, 130...","[10, 80, 100, 300, 20, 30, 40, 50, 60, 70, 130...",836
3,76561198042412488,"[300, 240, 220, 320, 360, 4300, 4800, 4000, 61...","[300, 240, 220, 320, 360, 4300, 4800, 4000, 61...",881
4,76561198119605821,"[47870, 108600, 550, 271590, 331470, 381210, 2...","[47870, 108600, 550, 271590, 331470, 381210, 2...",20


#### 3.2.2 Average Purchase Price

This feature requires looking up the `purchases` dataset with the price dataset we calculated in 2.2, then saved in 2.3.

For each game in a player's library, we lookup the `price` dataset for the base price of each game to compute the average value of the library.

As there are approximately 3.6k unique games in player libraries that don't exist in the `prices` dataset, we will treat those games with a price of $0. However, we will also track price coverage of each player, to know how much of their library could be looked up.

In [ ]:
price_dict = dict(zip(price_features['gameid'], price_features['base_price']))
price_keys = set(price_dict.keys())

avg_prices = []
price_coverage=[] # stores a percentage of the player's lib that was looked up

# iterate over all players
for _, row in tqdm(df_purchases.iterrows(), total=len(df_purchases), desc="Getting avg prices"):
    library = row['library_list']
    num_games = len(library)
    if num_games == 0:
      avg_prices.append(0.0)
      price_coverage.append(0.0)
      continue
    known_prices = [price_dict.get(gid, 0.0) for gid in library if gid in price_keys]
    avg_price = np.mean(known_prices) if known_prices else 0.0
    avg_prices.append(avg_price)
    price_coverage.append(len(known_prices) / num_games)

df_purchases['avg_purchase_price'] = avg_prices
df_purchases['price_coverage'] = price_coverage

display(df_purchases.head())
print(f'Brute force stats')
print(f"Mean of avg_purchase_price across all players: {df_purchases['avg_purchase_price'].mean():.2f}")
print(f"Players with 0 avg price: {(df_purchases['avg_purchase_price']==0).sum():,}\n")


Getting avg prices: 100%|██████████| 102548/102548 [00:13<00:00, 7420.16it/s]


,playerid,library,library_list,library_size,avg_purchase_price,price_coverage
0,76561198060698936,"[60, 1670, 3830, 1600, 2900, 2910, 2920, 4800,...","[60, 1670, 3830, 1600, 2900, 2910, 2920, 4800,...",3340,15.058132,0.929341
1,76561198287452552,"[10, 80, 100, 240, 2990, 6880, 6910, 6920, 698...","[10, 80, 100, 240, 2990, 6880, 6910, 6920, 698...",476,19.027540,0.930672
2,76561198040436563,"[10, 80, 100, 300, 20, 30, 40, 50, 60, 70, 130...","[10, 80, 100, 300, 20, 30, 40, 50, 60, 70, 130...",836,21.927873,0.888756
3,76561198042412488,"[300, 240, 220, 320, 360, 4300, 4800, 4000, 61...","[300, 240, 220, 320, 360, 4300, 4800, 4000, 61...",881,17.918178,0.884222
4,76561198119605821,"[47870, 108600, 550, 271590, 331470, 381210, 2...","[47870, 108600, 550, 271590, 331470, 381210, 2...",20,22.494500,1.000000


Brute force stats
Mean of avg_purchase_price across all players: 5.18
Players with 0 avg price: 57,930



In [ ]:
if ENVIRONMENT == 'colab':
    drive.mount('/content/drive')

df_purchases.to_csv(PURCHASE_FEATURES_PATH, index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
